In [7]:
import os
from dotenv import load_dotenv
import nest_asyncio
import pandas as pd
from tqdm.notebook import tqdm

# Load environment variables from .env file
load_dotenv()

nest_asyncio.apply()

In [8]:
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

model = 'azure_openai'

# for Azure OpenAI model
api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
gpt_api_version = os.getenv('AZURE_GPT_API_VERSION')
embedding_api_version = os.getenv('AZURE_EMBEDDING_API_VERSION')

if(model == 'local'):
    Settings.embed_model = embed_model
    Settings.llm = llm
elif(model == 'azure_openai'):
    llm = AzureOpenAI(
        model="gpt-4o",
        deployment_name="gpt-4o",
        api_key=api_key,
        azure_endpoint=azure_endpoint,
        api_version=gpt_api_version,
        temperature=0.1,  # Lower temperature for more consistent decision-making
        timeout=60,  # Increased timeout value
    )
    embed_model = AzureOpenAIEmbedding(
        model="text-embedding-ada-002",
        deployment_name="text-embedding-ada-002",
        api_key=api_key,
        azure_endpoint=azure_endpoint,
        api_version=embedding_api_version,
    )
    Settings.llm = llm
    Settings.embed_model = embed_model
elif(model == 'openai'):
    llm = OpenAI(model="gpt-4o")
    Settings.llm = llm
    Settings.embed_model = OpenAIEmbedding(
        model="text-embedding-3-small", embed_batch_size=256
    )

In [9]:

from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.workflow import Context, InputRequiredEvent, HumanResponseEvent
#from llama_index.core.events import InputRequiredEvent, HumanResponseEvent
from typing import Dict, List, Any, Tuple
import asyncio
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("RankRejectAgent")

# Define tools for the agents
async def evaluate_hazard_classification(ctx: Context, property_data: Dict[str, Any]) -> str:
    """
    Evaluates the hazard classification of a property based on building type, 
    construction materials, and occupancy.
    
    Args:
        property_data: Dictionary containing property details
        
    Returns:
        A description of the hazard assessment with score (1-5)
    """
    logger.info("Starting hazard classification assessment")
    current_state = await ctx.get("state")
    
    # Simulate hazard classification logic
    building_type = property_data.get("building_type", "")
    construction = property_data.get("construction", "")
    occupancy = property_data.get("occupancy", "")
    
    logger.info(f"Analyzing property: {building_type} building with {construction} construction")
    
    # Sample logic - would be more sophisticated in production
    hazard_factors = {
        "building_type": {
            "Office": 1,
            "Retail": 2,
            "Manufacturing": 3,
            "Warehouse": 3,
            "Heavy Industrial": 4,
            "Nightclub": 5
        },
        "construction": {
            "Concrete": 1,
            "Steel Frame": 2,
            "Brick": 2,
            "Wood Frame": 4,
            "Mixed": 3
        }
    }

    bt_score = hazard_factors["building_type"].get(building_type, 3)
    const_score = 3  # Default score
    for material, score in hazard_factors["construction"].items():
        if material.lower() in construction.lower():
            const_score = score
            break

     # Calculate overall hazard score
    hazard_score = (bt_score + const_score) / 2
    logger.info(f"Calculated hazard score: {hazard_score}")
    
    # Store the result in context
    if "assessment_results" not in current_state:
        current_state["assessment_results"] = {}
    
    current_state["assessment_results"]["hazard"] = {
        "score": hazard_score,
        "building_type_assessment": f"{building_type}: Risk level {bt_score}",
        "construction_assessment": f"{construction}: Risk level {const_score}",
        "occupancy_details": occupancy
    }
    
    # FIXED: Track completion as a list instead of a set for better serialization
    if "completed_assessments" not in current_state:
        current_state["completed_assessments"] = []
        
    if "hazard" not in current_state["completed_assessments"]:
        current_state["completed_assessments"].append("hazard")
    
    logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
    await ctx.set("state", current_state)
    
    logger.info("Hazard assessment completed and stored in context")
    return f"Hazard assessment completed. Overall hazard score: {hazard_score:.1f}/5.0"

async def evaluate_vulnerability(ctx: Context, security_data: Dict[str, Any]) -> str:
    """
    Evaluates the vulnerability of a property based on security systems,
    protective measures, and other safety features.
    
    Args:
        security_data: Dictionary containing security details
        
    Returns:
        A description of the vulnerability assessment with score (1-5)
    """
    logger.info("Starting vulnerability assessment")
    current_state = await ctx.get("state")
    
    # Simulate vulnerability assessment
    has_sprinklers = security_data.get("sprinklers", False)
    alarm_system = security_data.get("alarm_system", "None")
    
    logger.info(f"Security features: Sprinklers={has_sprinklers}, Alarm={alarm_system}")
    
    # Basic scoring
    sprinkler_score = 1 if has_sprinklers else 4
    
    alarm_scores = {
        "None": 5,
        "Local": 3,
        "Monitored": 2,
        "Grade A - 24hr Monitored": 1
    }
    alarm_score = alarm_scores.get(alarm_system, 3)
    
    # Calculate overall vulnerability score
    vulnerability_score = (sprinkler_score + alarm_score) / 2
    logger.info(f"Calculated vulnerability score: {vulnerability_score}")

    # Store the result
    if "assessment_results" not in current_state:
        current_state["assessment_results"] = {}
    
    current_state["assessment_results"]["vulnerability"] = {
        "score": vulnerability_score,
        "sprinkler_assessment": f"Sprinklers: {'Present' if has_sprinklers else 'Absent'}, Risk level {sprinkler_score}",
        "alarm_assessment": f"Alarm: {alarm_system}, Risk level {alarm_score}"
    }
    
    # FIXED: Track completion as a list instead of a set for better serialization
    if "completed_assessments" not in current_state:
        current_state["completed_assessments"] = []
        
    if "vulnerability" not in current_state["completed_assessments"]:
        current_state["completed_assessments"].append("vulnerability")
    
    logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
    await ctx.set("state", current_state)
    
    logger.info("Vulnerability assessment completed and stored in context")
    return f"Vulnerability assessment completed. Overall vulnerability score: {vulnerability_score:.1f}/5.0"

async def evaluate_cat_modeling(ctx: Context, location_data: Dict[str, Any]) -> str:
    """
    Evaluates the catastrophe risk based on location, flood zones, 
    earthquake potential, etc.
    
    Args:
        location_data: Dictionary containing location details
        
    Returns:
        A description of the CAT modeling assessment with score (1-5)
    """
    logger.info("Starting CAT modeling assessment")
    current_state = await ctx.get("state")
    
    # In production, this would call specialized CAT modeling services
    address = location_data.get("address", "")
    logger.info(f"Analyzing location: {address}")
    
    # Mock implementation - would use real data in production
    # Simulating a flood zone and earthquake risk evaluation
    if "flood" in address.lower() or "coastal" in address.lower():
        flood_risk = 4.5
    else:
        flood_risk = 2.0
    
    # Simple geographic rules - would use proper services in production
    geo_mapping = {
        "california": 4.5,  # High earthquake risk
        "florida": 4.0,     # Hurricane risk
        "texas": 3.5,       # Multiple hazards
        "new york": 3.0,
        "london": 2.0,
        "birmingham": 1.5,
        "manchester": 2.0
    }
    
    earthquake_risk = 1.0  # Default low risk
    for region, risk in geo_mapping.items():
        if region.lower() in address.lower():
            earthquake_risk = risk
            break
    
    # Calculate overall CAT score
    cat_score = max(flood_risk, earthquake_risk)  # Taking worst-case scenario
    logger.info(f"Calculated CAT score: {cat_score} (flood_risk: {flood_risk}, earthquake_risk: {earthquake_risk})")
    
    # Store the result
    if "assessment_results" not in current_state:
        current_state["assessment_results"] = {}
    
    current_state["assessment_results"]["cat_modeling"] = {
        "score": cat_score,
        "flood_risk_assessment": f"Flood risk level: {flood_risk:.1f}/5.0",
        "earthquake_risk_assessment": f"Earthquake/natural disaster risk: {earthquake_risk:.1f}/5.0",
        "location_analyzed": address
    }
    
    # FIXED: Track completion as a list instead of a set for better serialization
    if "completed_assessments" not in current_state:
        current_state["completed_assessments"] = []
        
    if "cat_modeling" not in current_state["completed_assessments"]:
        current_state["completed_assessments"].append("cat_modeling")
    
    logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
    await ctx.set("state", current_state)
    
    logger.info("CAT modeling assessment completed and stored in context")
    return f"CAT modeling assessment completed. Overall CAT risk score: {cat_score:.1f}/5.0"

async def check_guidelines_compliance(ctx: Context, coverage_data: Dict[str, Any]) -> str:
    """
    Checks if the submission complies with underwriting guidelines for 
    limits, deductibles, and pricing factors.
    
    Args:
        coverage_data: Dictionary containing coverage details
        
    Returns:
        A description of the compliance assessment
    """
    logger.info("Starting guidelines compliance assessment")
    current_state = await ctx.get("state")
    
    # Mock guideline rules - would be loaded from database in production
    guidelines = {
        "building_value": {
            "min": 500000,
            "max": 10000000
        },
        "deductible": {
            "min_percentage": 0.005,  # Minimum deductible should be 0.5% of building value
            "preferred_percentage": 0.01  # Preferred deductible is 1% of building value
        },
        "business_interruption": {
            "max_ratio": 2.0  # BI shouldn't be more than 2x building value
        }
    }
    
    # Extract values
    building_value = coverage_data.get("building_value", 0)
    deductible = coverage_data.get("deductible", 0)
    business_interruption = coverage_data.get("business_interruption", 0)
    
    logger.info(f"Analyzing coverage: Building value={building_value}, Deductible={deductible}, Business interruption={business_interruption}")
    
    # Check compliance
    compliance_issues = []
    
    if building_value < guidelines["building_value"]["min"]:
        compliance_issues.append(f"Building value too low: £{building_value:,.2f} (minimum: £{guidelines['building_value']['min']:,.2f})")
    
    if building_value > guidelines["building_value"]["max"]:
        compliance_issues.append(f"Building value exceeds guideline maximum: £{building_value:,.2f} (maximum: £{guidelines['building_value']['max']:,.2f})")
    
    min_required_deductible = building_value * guidelines["deductible"]["min_percentage"]
    if deductible < min_required_deductible:
        compliance_issues.append(f"Deductible too low: £{deductible:,.2f} (minimum required: £{min_required_deductible:,.2f})")
    
    if business_interruption > building_value * guidelines["business_interruption"]["max_ratio"]:
        compliance_issues.append(f"Business interruption coverage exceeds guidelines: £{business_interruption:,.2f} (maximum allowed: £{building_value * guidelines['business_interruption']['max_ratio']:,.2f})")
    
    # Calculate compliance score - lower is better
    if len(compliance_issues) == 0:
        compliance_score = 1.0
    else:
        compliance_score = 1.0 + len(compliance_issues)
    
    logger.info(f"Compliance assessment - Issues: {len(compliance_issues)}, Score: {compliance_score}")
    
    # Store the result
    if "assessment_results" not in current_state:
        current_state["assessment_results"] = {}
    
    current_state["assessment_results"]["guidelines_compliance"] = {
        "score": compliance_score,
        "compliant": len(compliance_issues) == 0,
        "issues": compliance_issues,
        "coverage_reviewed": {
            "building_value": building_value,
            "deductible": deductible,
            "business_interruption": business_interruption
        }
    }
    
    # FIXED: Track completion as a list instead of a set for better serialization
    if "completed_assessments" not in current_state:
        current_state["completed_assessments"] = []
        
    if "guidelines_compliance" not in current_state["completed_assessments"]:
        current_state["completed_assessments"].append("guidelines_compliance")
    
    logger.info(f"Updated completed_assessments: {current_state['completed_assessments']}")
    await ctx.set("state", current_state)
    
    logger.info("Guidelines compliance assessment completed and stored in context")
    if len(compliance_issues) == 0:
        return "Guidelines compliance check passed. No issues found."
    else:
        return f"Guidelines compliance check completed with {len(compliance_issues)} issue(s): " + "; ".join(compliance_issues)
    
async def make_decision(ctx: Context) -> str:
    """
    Checks if all assessments are complete, then analyzes results and makes a final decision.
    If confidence is low, requests human review.
    
    Returns:
        Decision outcome with explanation
    """
    logger.info("Starting decision making process")
    current_state = await ctx.get("state")
    
    # FIXED: Check if all assessments are complete using a list instead of a set
    completed_assessments = current_state.get("completed_assessments", [])
    required_assessments = ["hazard", "vulnerability", "cat_modeling", "guidelines_compliance"]
    
    missing_assessments = [a for a in required_assessments if a not in completed_assessments]
    
    if missing_assessments:
        logger.info(f"Cannot make decision yet. Missing assessments: {missing_assessments}")
        return f"Cannot make decision yet. The following assessments are still pending: {', '.join(missing_assessments)}"
    
    # Check if we already have a final decision (to prevent infinite loops)
    if current_state.get("decision", {}).get("final", False):
        decision = current_state.get("decision", {}).get("outcome", "UNKNOWN")
        reason = current_state.get("decision", {}).get("reason", "No reason provided")
        confidence = current_state.get("decision", {}).get("confidence", 0.0)
        
        decision_messages = {
            "PROCEED_TO_QUOTATION": "Submission approved to proceed to quotation.",
            "RECOMMEND_SURVEYOR": "Recommend surveyor assessment before proceeding.",
            "REJECT": "Submission rejected."
        }
        
        logger.info(f"Using existing final decision: {decision}")
        return f"Final decision already made: {decision_messages.get(decision, decision)}. Reason: {reason}. Confidence: {confidence:.2f}"
    
    # Continue with decision making if all assessments are complete
    logger.info("All assessments are complete. Proceeding with decision making.")
    assessment_results = current_state.get("assessment_results", {})
    submission = current_state.get("submission_data", {})
    
    # Gather scores from assessments
    hazard_score = assessment_results.get("hazard", {}).get("score", 3.0)
    vulnerability_score = assessment_results.get("vulnerability", {}).get("score", 3.0)
    cat_score = assessment_results.get("cat_modeling", {}).get("score", 3.0)
    compliance_score = assessment_results.get("guidelines_compliance", {}).get("score", 3.0)
    is_compliant = assessment_results.get("guidelines_compliance", {}).get("compliant", False)
    
    logger.info(f"Assessment scores - Hazard: {hazard_score}, Vulnerability: {vulnerability_score}, CAT: {cat_score}, Compliance: {compliance_score}")
    
    # Calculate composite risk score (weighted average)
    weights = {
        "hazard": 0.3,
        "vulnerability": 0.2,
        "cat": 0.3,
        "compliance": 0.2
    }
    
    composite_score = (
        hazard_score * weights["hazard"] +
        vulnerability_score * weights["vulnerability"] +
        cat_score * weights["cat"] +
        compliance_score * weights["compliance"]
    )
    
    logger.info(f"Calculated composite risk score: {composite_score}")
    
    # Decision logic
    if not is_compliant:
        decision = "REJECT"
        reason = "Submission does not comply with underwriting guidelines"
        confidence = 0.95
    elif composite_score <= 2.5:
        decision = "PROCEED_TO_QUOTATION"
        reason = "Risk profile is within acceptable parameters"
        confidence = min(1.0, max(0.0, 1.0 - (composite_score / 5.0)))
    elif composite_score <= 3.5:
        decision = "RECOMMEND_SURVEYOR"
        reason = "Risk profile requires additional assessment"
        confidence = 0.8
    else:
        decision = "REJECT"
        reason = "Risk profile exceeds acceptable parameters"
        confidence = min(1.0, max(0.0, composite_score / 5.0 - 0.2))
    
    logger.info(f"Initial decision: {decision}, Reason: {reason}, Confidence: {confidence:.2f}")
    
    # Human-in-the-loop for decisions with low confidence or rejections
    if confidence < 0.7 or decision == "REJECT":
        # Check if we already received human feedback (to prevent loops)
        if current_state.get("human_feedback_received", False):
            logger.info("Human feedback was already received, using it to finalize decision")
            feedback_comment = current_state.get("human_feedback_comment", "No feedback provided")
        else:
            logger.info(f"Decision requires human review. Confidence: {confidence:.2f}")
            
            # Request human feedback
            feedback_comment = await request_human_feedback(ctx, 
                f"Please review decision: {decision} for submission {submission.get('submission_id', 'Unknown')}. Confidence: {confidence:.2f}")
            
            # Store the feedback
            current_state["human_feedback_received"] = True
            current_state["human_feedback_comment"] = feedback_comment
            
            await ctx.set("state", current_state)
            logger.info(f"Human feedback received: {feedback_comment}")
        
        # Modify the decision based on feedback if needed
        if "approve" in feedback_comment.lower() or "proceed" in feedback_comment.lower():
            # Override the decision if human approves
            if decision == "REJECT":
                decision = "PROCEED_TO_QUOTATION" 
                reason = f"Risk profile approved by human reviewer despite system assessment"
                confidence = 0.85  # Human override increases confidence
                logger.info("Decision changed to PROCEED_TO_QUOTATION based on human feedback")
        elif "surveyor" in feedback_comment.lower() or "survey" in feedback_comment.lower():
            # Change to surveyor recommendation if that's what human suggests
            if decision != "RECOMMEND_SURVEYOR":
                decision = "RECOMMEND_SURVEYOR"
                reason = f"Human reviewer recommended additional assessment"
                confidence = 0.9  # Human override increases confidence
                logger.info("Decision changed to RECOMMEND_SURVEYOR based on human feedback")
        elif "reject" in feedback_comment.lower() or "decline" in feedback_comment.lower():
            # Confirm rejection if human agrees
            if decision != "REJECT":
                decision = "REJECT"
                reason = f"Human reviewer recommended rejection"
                confidence = 0.95  # Human override increases confidence
                logger.info("Decision changed to REJECT based on human feedback")
    
    # Store the final decision
    current_state["decision"] = {
        "outcome": decision,
        "reason": reason,
        "composite_score": composite_score,
        "confidence": confidence,
        "requires_human_review": confidence < 0.8,
        "human_reviewed": current_state.get("human_feedback_received", False),
        "assessment_summary": {
            "hazard_score": hazard_score,
            "vulnerability_score": vulnerability_score,
            "cat_score": cat_score,
            "compliance_score": compliance_score,
            "is_compliant": is_compliant
        },
        "final": True  # Mark this decision as final to prevent loops
    }
    
    await ctx.set("state", current_state)
    
    decision_messages = {
        "PROCEED_TO_QUOTATION": "Submission approved to proceed to quotation.",
        "RECOMMEND_SURVEYOR": "Recommend surveyor assessment before proceeding.",
        "REJECT": "Submission rejected."
    }
    
    logger.info("Decision made and stored in context")
    return f"Decision: {decision_messages[decision]} Reason: {reason}. Confidence: {confidence:.2f}"


async def send_notification(ctx: Context) -> str:
    """
    Formats and sends notification email based on the decision.
    
    Returns:
        Confirmation of notification sent
    """
    logger.info("Starting notification preparation")
    current_state = await ctx.get("state")
    decision = current_state.get("decision", {})

    # Check if this is a final decision
    if not decision.get("final", False):
        return "Cannot send notification for a non-final decision. Decision requires human review first."

    # Get the decision outcome and reason
    submission = current_state.get("submission_data", {})
    
    # In production, this would use Azure Communication Services or similar
    
    # Format email content based on decision outcome
    outcome = decision.get("outcome", "UNKNOWN")
    reason = decision.get("reason", "No reason provided")
    
    logger.info(f"Preparing notification for decision: {outcome}")
    
    # Basic email templates
    email_templates = {
        "PROCEED_TO_QUOTATION": """
Subject: Submission {submission_id} Approved for Quotation

Dear Distribution Team,

The submission for {insured_name} (ID: {submission_id}) has been reviewed and approved to proceed to quotation.

Risk Assessment Summary:
- Hazard Score: {hazard_score}/5.0
- Vulnerability Score: {vulnerability_score}/5.0
- CAT Risk Score: {cat_score}/5.0
- Compliance: {compliance}

Decision: Proceed to Quotation
Confidence: {confidence:.0%}

Please proceed with the quotation process.

Regards,
Underwriting AI Assistant
""",
        "RECOMMEND_SURVEYOR": """
Subject: Submission {submission_id} Requires Surveyor Assessment

Dear Distribution Team,

The submission for {insured_name} (ID: {submission_id}) has been reviewed and requires a surveyor assessment before proceeding.

Risk Assessment Summary:
- Hazard Score: {hazard_score}/5.0
- Vulnerability Score: {vulnerability_score}/5.0
- CAT Risk Score: {cat_score}/5.0
- Compliance: {compliance}

Reason for surveyor recommendation: {reason}
Confidence: {confidence:.0%}

Please arrange for a risk assessment survey.

Regards,
Underwriting AI Assistant
""",
        "REJECT": """
Subject: Submission {submission_id} Rejected

Dear Distribution Team,

The submission for {insured_name} (ID: {submission_id}) has been reviewed and cannot proceed.

Risk Assessment Summary:
- Hazard Score: {hazard_score}/5.0
- Vulnerability Score: {vulnerability_score}/5.0
- CAT Risk Score: {cat_score}/5.0
- Compliance: {compliance}

Reason for rejection: {reason}
Confidence: {confidence:.0%}

If you have additional information that might change this assessment, please provide it.

Regards,
Underwriting AI Assistant
"""
    }
    
    # Get template for current decision
    template = email_templates.get(outcome, "Unknown decision type")
    
    # Fill in template variables
    assessment_summary = decision.get("assessment_summary", {})
    email_content = template.format(
        submission_id=submission.get("submission_id", "Unknown"),
        insured_name=submission.get("insured_name", "Unknown"),
        hazard_score=assessment_summary.get("hazard_score", 0),
        vulnerability_score=assessment_summary.get("vulnerability_score", 0),
        cat_score=assessment_summary.get("cat_score", 0),
        compliance="Compliant" if assessment_summary.get("is_compliant", False) else "Non-compliant",
        reason=reason,
        confidence=decision.get("confidence", 0.0)
    )
    
    # Store the notification in the state
    current_state["notification"] = {
        "recipient": "distribution_team@company.com",
        "content": email_content,
        "sent": True,
        "timestamp": "2025-04-17T10:30:00Z"  # This would be actual timestamp in production
    }
    
    # Mark workflow as complete
    current_state["workflow_complete"] = True
    await ctx.set("state", current_state)
    
    # In production, this would actually send the email
    logger.info(f"Email notification prepared for distribution team regarding submission {submission.get('submission_id', 'Unknown')}")
    
    return f"Notification email prepared and ready to send for decision: {outcome}"

async def request_human_feedback(ctx: Context, question: str = "") -> str:
    """
    Requests human feedback using LlamaIndex's event system.
    
    Args:
        ctx: Context object
        question: Question to ask the human reviewer
        
    Returns:
        Human feedback response
    """
    logger.info(f"Requesting human feedback: {question}")
    current_state = await ctx.get("state")
    
    # Get decision and submission details for context
    decision = current_state.get("decision", {})
    submission = current_state.get("submission_data", {})
    assessment_results = current_state.get("assessment_results", {})
    
    # Create a detailed prompt for the human reviewer
    detailed_question = f"""
HUMAN REVIEW REQUIRED:
{question}

Submission: {submission.get('submission_id')} - {submission.get('insured_name')}
Property: {submission.get('property_details', {}).get('building_type')} at {submission.get('property_details', {}).get('address')}

Assessment Results:
- Hazard: {assessment_results.get('hazard', {}).get('score', 0)}/5.0
- Vulnerability: {assessment_results.get('vulnerability', {}).get('score', 0)}/5.0
- CAT Risk: {assessment_results.get('cat_modeling', {}).get('score', 0)}/5.0
- Compliance: {'Compliant' if assessment_results.get('guidelines_compliance', {}).get('compliant', False) else 'Non-compliant'}

System Decision: {decision.get('outcome', 'Unknown')}
Reason: {decision.get('reason', 'No reason provided')}
Confidence: {decision.get('confidence', 0):.0%}

Please review and provide feedback. Type 'approve' to confirm the decision, or provide specific guidance:
"""
    
    # Use LlamaIndex's wait_for_event to get human input
    response = await ctx.wait_for_event(
        HumanResponseEvent,
        waiter_id=question,
        waiter_event=InputRequiredEvent(
            prefix=detailed_question,
            user_name="Underwriter",
        ),
        requirements={"user_name": "Underwriter"},
    )
    
    # Record the feedback in the state
    if "human_feedback" not in current_state:
        current_state["human_feedback"] = {}
    
    current_state["human_feedback"]["requested"] = True
    current_state["human_feedback"]["timestamp"] = "2025-04-17T10:45:00Z"  # This would be actual timestamp in production
    current_state["human_feedback"]["comment"] = response.response
    current_state["human_feedback"]["status"] = "completed"
    
    await ctx.set("state", current_state)
    
    return response.response

async def process_human_feedback(ctx: Context, approval: bool, feedback: str = "") -> str:
    """
    Processes human feedback on the decision.
    
    Args:
        approval: Whether the human approves the decision
        feedback: Optional feedback from the human reviewer
        
    Returns:
        Confirmation of feedback processed
    """
    current_state = await ctx.get("state")
    
    # Update the human feedback status
    if "human_feedback" not in current_state:
        current_state["human_feedback"] = {}
    
    current_state["human_feedback"]["status"] = "completed"
    current_state["human_feedback"]["approved"] = approval
    current_state["human_feedback"]["feedback"] = feedback
    current_state["human_feedback"]["processed_timestamp"] = "2025-04-14T10:40:00Z"  # This would be actual timestamp
    
    # If not approved, record for model improvement
    if not approval:
        if "model_improvement_data" not in current_state:
            current_state["model_improvement_data"] = []
        
        current_state["model_improvement_data"].append({
            "original_decision": current_state.get("decision", {}).get("outcome", "UNKNOWN"),
            "feedback": feedback,
            "submission_data": current_state.get("submission_data", {}),
            "assessment_results": current_state.get("assessment_results", {})
        })
    
    await ctx.set("state", current_state)
    
    if approval:
        return "Human feedback processed: Decision approved."
    else:
        return f"Human feedback processed: Decision requires changes. Feedback: {feedback}"
    


In [10]:
# Define the agents with sequential handoff pattern
hazard_agent = FunctionAgent(
    name="HazardClassificationAgent",
    description="Evaluates the hazard classification of properties based on building type, construction, and occupancy.",
    system_prompt=(
        "You are a specialist in evaluating property hazard classifications for insurance purposes. "
        "You analyze building types, construction materials, and occupancy types to determine the risk level. "
        "Be thorough and precise in your assessments, using standard insurance industry criteria. "
        "After completing your assessment, hand off to the VulnerabilityAssessmentAgent."
    ),
    llm=llm,
    tools=[evaluate_hazard_classification],
    can_handoff_to=["VulnerabilityAssessmentAgent"]
)

vulnerability_agent = FunctionAgent(
    name="VulnerabilityAssessmentAgent",
    description="Evaluates property vulnerability based on security systems and protective measures.",
    system_prompt=(
        "You are a specialist in evaluating property vulnerabilities for insurance purposes. "
        "You assess security systems, fire protection measures, and other safety features to determine the risk level. "
        "Be thorough in considering all protective measures and their effectiveness. "
        "After completing your assessment, hand off to the CATModelingAgent."
    ),
    llm=llm,
    tools=[evaluate_vulnerability],
    can_handoff_to=["CATModelingAgent"]
)

cat_modeling_agent = FunctionAgent(
    name="CATModelingAgent",
    description="Evaluates catastrophe risks based on geographical location and environmental factors.",
    system_prompt=(
        "You are a specialist in catastrophe risk modeling for insurance purposes. "
        "You analyze geographical locations to assess risks from natural disasters like floods, earthquakes, and storms. "
        "Use precise geographical data and historical patterns to determine risk levels. "
        "After completing your assessment, hand off to the GuidelinesComplianceAgent."
    ),
    llm=llm,
    tools=[evaluate_cat_modeling],
    can_handoff_to=["GuidelinesComplianceAgent"]
)

guidelines_agent = FunctionAgent(
    name="GuidelinesComplianceAgent", 
    description="Checks if submissions comply with underwriting guidelines for limits, deductibles, and pricing factors.",
    system_prompt=(
        "You are a specialist in insurance underwriting guidelines compliance. "
        "You verify that submissions meet all required parameters for coverage limits, deductibles, and other factors. "
        "Be precise in identifying any deviations from established guidelines. "
        "After completing your assessment, hand off to the DecisionAgent."
    ),
    llm=llm,
    tools=[check_guidelines_compliance],
    can_handoff_to=["DecisionAgent"]
)

decision_agent = FunctionAgent(
    name="DecisionAgent",
    description="Analyzes all assessment results and makes the final underwriting decision.",
    system_prompt=(
        "You are a decision specialist for insurance underwriting. "
        "You analyze assessment results from multiple domains to determine whether to proceed with quotation, "
        "recommend a surveyor, or reject a submission. "
        "First check that all required assessments (hazard, vulnerability, cat modeling, guidelines compliance) "
        "have been completed before making your decision. "
        "Balance all risk factors to make the most appropriate decision. "
        "After making your decision, hand off to the CommunicationAgent."
    ),
    llm=llm,
    tools=[make_decision],
    can_handoff_to=["CommunicationAgent"]
)

communication_agent = FunctionAgent(
    name="CommunicationAgent",
    description="Formats and sends notifications based on the underwriting decision.",
    system_prompt=(
        "You are a communication specialist for insurance operations. "
        "You create clear, appropriate notifications to inform stakeholders about underwriting decisions. "
        "Ensure all relevant information is included in the appropriate format. "
        "After sending the notification, the decision process is complete and no further action is needed."
    ),
    llm=llm,
    tools=[send_notification]
)

supervisor_agent = FunctionAgent(
    name="SupervisorAgent",
    description="Orchestrates the overall rank and reject workflow, managing state and decision processes.",
    system_prompt=(
        "You are the supervisor for the insurance submission rank and reject workflow. "
        "You coordinate multiple specialized agents, maintain the overall state, and ensure the process runs smoothly. "
        "Delegate tasks to appropriate specialists and synthesize their results."
    ),
    llm=llm,
    tools=[],
    can_handoff_to=["HazardClassificationAgent"]
)

In [11]:
example_submission = {
    "submission_id": "SUB20250414001",
    "broker_name": "ABC Insurance Brokers",
    "insured_name": "Acme Manufacturing Ltd",
    "property_details": {
        "address": "123 Industrial Way, Birmingham, B6 4BD",
        "building_type": "Manufacturing",
        "construction": "Steel Frame with Brick",
        "year_built": 1995,
        "area_sqm": 5000,
        "stories": 2,
        "occupancy": "Light Manufacturing - Electronics",
        "sprinklers": True,
        "alarm_system": "Grade A - 24hr Monitored"
    },
    "coverage": {
        "building_value": 4500000.00,
        "contents_value": 2000000.00,
        "business_interruption": 3000000.00,
        "deductible": 25000.00
    },
    "compliance_status": "PASSED",
    "client_history": {
        "claims_count": 1,
        "previous_policies": 2,
        "risk_score": 68
    }
}

In [12]:
def create_insurance_workflow(submission_data=None):
    """
    Create a streamlined workflow for insurance submission processing
    """
    if submission_data is None:
        submission_data = example_submission
        
    # Create the workflow with the 6 essential agents
    workflow = AgentWorkflow(
        agents=[
            hazard_agent,
            vulnerability_agent,
            cat_modeling_agent,
            guidelines_agent,
            decision_agent,
            communication_agent
        ],
        # Start with hazard agent
        root_agent=hazard_agent.name,
        initial_state={
            "submission_data": submission_data,
            "assessment_results": {},
            # FIXED: Initialize with an empty list instead of a set
            "completed_assessments": [],
            "decision": {},
            "notification": {},
            "workflow_complete": False
        }
    )
    
    return workflow

In [13]:
async def run_workflow(submission_data=None):
    """Run the multi-agent workflow with human-in-the-loop capability"""
    workflow = create_insurance_workflow(submission_data)
    
    logger.info("Starting workflow execution...")
    
    # Run the workflow with a more explicit message
    handler = workflow.run(
        user_msg=(
            "Please process this insurance submission through the full assessment workflow. "
            "Begin with the hazard classification, then evaluate vulnerability, "
            "perform CAT modeling, check guidelines compliance, "
            "make a decision, and finally send the appropriate notification."
        )
    )
    
    # Stream the output to see progress
    current_agent = None
    
    async for event in handler.stream_events():
        # Check if workflow is complete after each step
        current_state = await handler.ctx.get("state")
        if current_state.get("workflow_complete", False):
            logger.info("Workflow has been marked as complete, stopping further processing.")
            break
            
        if hasattr(event, "current_agent_name") and event.current_agent_name != current_agent:
            current_agent = event.current_agent_name
            logger.info(f"==== Agent: {current_agent} ====")
            
        # Handle human input requests
        if isinstance(event, InputRequiredEvent):
            # In a real application, this could be a GUI input, email notification, etc.
            print("\n" + "="*50)
            print("\nHUMAN REVIEW REQUIRED")
            print("="*50)
            response = input(event.prefix + "\n> ")
            print("="*50 + "\n")
            
            # Send the human response back to the workflow
            handler.ctx.send_event(
                HumanResponseEvent(
                    response=response,
                    user_name=event.user_name,
                )
            )
        
        # Check if a final decision has been made
        if current_state.get("decision", {}).get("final", False) and current_agent == "CommunicationAgent":
            logger.info("Final decision has been made and notification sent. Workflow will complete.")
            current_state["workflow_complete"] = True
            await handler.ctx.set("state", current_state)
    
    # Get the final state
    final_state = await handler.ctx.get("state")
    
    # Log the results
    logger.info("Workflow execution completed.")
    logger.info(f"Decision: {final_state.get('decision', {}).get('outcome', 'No decision')}")
    logger.info(f"Reason: {final_state.get('decision', {}).get('reason', 'No reason provided')}")
    
    # Return the final state
    return final_state

In [14]:
def main():
    """Main execution function"""
    logger.info("Starting the insurance submission assessment system...")
    
    try:
        # Run the workflow
        final_state = asyncio.run(run_workflow())
        
        # Print the decision details
        decision = final_state.get("decision", {})
        
        print("\n====== INSURANCE SUBMISSION ASSESSMENT RESULTS ======")
        print(f"Submission ID: {final_state.get('submission_data', {}).get('submission_id', 'Unknown')}")
        print(f"Insured: {final_state.get('submission_data', {}).get('insured_name', 'Unknown')}")
        print("\nCOMPLETED ASSESSMENTS:")
        print(f"- {', '.join(final_state.get('completed_assessments', []))}")
        print("\nRISK ASSESSMENT SUMMARY:")
        
        assessment_summary = decision.get("assessment_summary", {})
        print(f"- Hazard Score: {assessment_summary.get('hazard_score', 0):.1f}/5.0")
        print(f"- Vulnerability Score: {assessment_summary.get('vulnerability_score', 0):.1f}/5.0")
        print(f"- CAT Risk Score: {assessment_summary.get('cat_score', 0):.1f}/5.0")
        print(f"- Compliance: {'Compliant' if assessment_summary.get('is_compliant', False) else 'Non-compliant'}")
        
        print(f"\nDECISION: {decision.get('outcome', 'Unknown')}")
        print(f"Reason: {decision.get('reason', 'No reason provided')}")
        print(f"Confidence: {decision.get('confidence', 0):.0%}")
        
        # Show human feedback if available
        human_feedback = final_state.get("human_feedback", {})
        if human_feedback:
            print("\nHUMAN FEEDBACK:")
            print(f"Status: {human_feedback.get('status', 'Not provided')}")
            print(f"Comment: {human_feedback.get('comment', 'No comment')}")
        
        # Show notification summary
        notification = final_state.get("notification", {})
        if notification:
            print("\nNOTIFICATION SENT:")
            print(f"Recipient: {notification.get('recipient', 'Unknown')}")
            print(f"Timestamp: {notification.get('timestamp', 'Unknown')}")
            print("Email Content Preview: ")
            content = notification.get("content", "")
            print(content[:min(200, len(content))] + "..." if len(content) > 200 else content)
        
        print("\n===========================================")
        
        return final_state
        
    except Exception as e:
        logger.error(f"Error running workflow: {str(e)}")
        logger.error(f"Error type: {type(e)}")
        raise

if __name__ == "__main__":
    main()

2025-04-17 12:19:13,106 - RankRejectAgent - INFO - Starting the insurance submission assessment system...
2025-04-17 12:19:13,108 - RankRejectAgent - INFO - Starting workflow execution...
2025-04-17 12:19:13,285 - RankRejectAgent - INFO - ==== Agent: HazardClassificationAgent ====
2025-04-17 12:19:14,541 - httpx - INFO - HTTP Request: POST https://uksouth-openai-shareable.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-04-17 12:19:14,590 - RankRejectAgent - INFO - Starting hazard classification assessment
2025-04-17 12:19:14,591 - RankRejectAgent - INFO - Analyzing property: Manufacturing building with Steel Frame with Brick construction
2025-04-17 12:19:14,591 - RankRejectAgent - INFO - Calculated hazard score: 2.5
2025-04-17 12:19:14,592 - RankRejectAgent - INFO - Updated completed_assessments: ['hazard']
2025-04-17 12:19:14,592 - RankRejectAgent - INFO - Hazard assessment completed and stored in context
2025-04-17 12:



HUMAN REVIEW REQUIRED


2025-04-17 12:20:09,147 - RankRejectAgent - INFO - Human feedback received: 
2025-04-17 12:20:09,148 - RankRejectAgent - INFO - Decision made and stored in context


2025-04-17 12:20:09,503 - httpx - INFO - HTTP Request: POST https://uksouth-openai-shareable.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-04-17 12:20:09,661 - RankRejectAgent - INFO - ==== Agent: CommunicationAgent ====
2025-04-17 12:20:09,662 - RankRejectAgent - INFO - Final decision has been made and notification sent. Workflow will complete.
2025-04-17 12:20:10,010 - httpx - INFO - HTTP Request: POST https://uksouth-openai-shareable.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-04-17 12:20:10,013 - RankRejectAgent - INFO - Workflow has been marked as complete, stopping further processing.
2025-04-17 12:20:10,014 - RankRejectAgent - INFO - Workflow execution completed.
2025-04-17 12:20:10,014 - RankRejectAgent - INFO - Decision: PROCEED_TO_QUOTATION
2025-04-17 12:20:10,015 - RankRejectAgent - INFO - Reason: Risk profile is within acceptable paramete


====== INSURANCE SUBMISSION ASSESSMENT RESULTS ======
Submission ID: SUB20250414001
Insured: Acme Manufacturing Ltd

COMPLETED ASSESSMENTS:
- hazard, vulnerability, cat_modeling, guidelines_compliance

RISK ASSESSMENT SUMMARY:
- Hazard Score: 2.5/5.0
- Vulnerability Score: 1.0/5.0
- CAT Risk Score: 2.0/5.0
- Compliance: Compliant

DECISION: PROCEED_TO_QUOTATION
Reason: Risk profile is within acceptable parameters
Confidence: 65%

HUMAN FEEDBACK:
Status: completed
Comment: 



2025-04-17 12:20:10,038 - RankRejectAgent - INFO - Starting notification preparation
2025-04-17 12:20:10,039 - RankRejectAgent - INFO - Preparing notification for decision: PROCEED_TO_QUOTATION
2025-04-17 12:20:10,040 - RankRejectAgent - INFO - Email notification prepared for distribution team regarding submission SUB20250414001
2025-04-17 12:20:10,436 - httpx - INFO - HTTP Request: POST https://uksouth-openai-shareable.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
